## E17 - ocr

google cloud platform에서 vision api 이용하기   
서비스 계정 및 인증키가 필요하다   
http://egloos.zum.com/mcchae/v/11342622   
https://console.cloud.google.com/   
my_google_api_key.json파일로 저장하기   

In [1]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
       print('\n"{}"'.format(text.description))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                 for vertex in text.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [2]:
# 다운받은 인증키 경로가 정확하게 지정되어 있어야 합니다. 
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

# 입력 이미지 경로를 지정해 주세요.
path = '/HOME/aiffel0042/aiffel/ocr_python/na.jpeg'  
# ex.) '~/aiffel/ocr_python/test_image.png'

# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
detect_text(path)

total 13176
drwxr-xr-x 3 aiffel0042 aiffel0042    4096  9월 17 20:30 '~'
-rw-r--r-- 1 aiffel0042 aiffel0042    1441  9월 25 15:42  _about.txt
-rw-r--r-- 1 aiffel0042 aiffel0042    9791  7월 29 10:03  E01.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   55628  8월 14 16:37  E01_study.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   20507  7월 31 10:26  E02.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   19271  8월 14 18:17  E02_study.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   18440  8월  5 10:21  E03.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   50152  8월 26 18:21  E03_study.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042  292083  8월  7 15:16  E04.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042  507549  8월 12 10:04  E05.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042  316809  8월 12 10:30  E05_study.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042  490931  8월 14 09:25  E06.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042 1457815  8월 28 01:42  E06_study.ipynb
-rw-r--r-- 1 aiffel0042 aiffel0042   29357  8월 20 10:50  E07.ipynb
-rw-r--r-- 1 aiffel0042 ai

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started